In [16]:
import pandas as pd

from tinydb import TinyDB, Query
from tinydb.storages import JSONStorage
from tinydb.middlewares import CachingMiddleware

from tqdm import tqdm_notebook as tqdm

In [17]:
symbol_df = pd.DataFrame(columns=['Symbol', 'Name', 'Symbol2', 'Currency', 'Exchange', 'Status'])

In [18]:
exchanges = {
    'FWB': 'ib_fwb_etfs.csv',
    'LSE': 'ib_lse_etfs.csv',
    'LSEETF': 'ib_lse-etf_etfs.csv',
    'IBIS': 'ib_xetra_etfs.csv'
}

In [19]:
for ex, file in exchanges.items():
    one_ex_data = pd.read_csv(file, delimiter=';')
    one_ex_data['Exchange'] = ex
    symbol_df = symbol_df.append(one_ex_data, ignore_index=True, sort=True)
symbol_df.describe()

,Currency,Exchange,Name,Status,Symbol,Symbol2
count,5323,5323,5323,0,5323,5323
unique,3,4,2284,0,3732,3955
top,EUR,LSE,WisdomTree US Equity Income UCITS ETF,NaN,STAW,3DES
freq,3024,1827,12,NaN,4,3


In [10]:
# Write to CSV
symbol_df.to_csv('symbol_database.csv', sep=';')

In [23]:
# Write list to textfile

# df = symbol_database[symbol_database['Symbol']!=symbol_database['Symbol2']]
df = symbol_df.loc[symbol_df['exchange']=='fwb', 'Symbol']
symbol_list = df.tolist()

file = open('symbollist.txt', 'w+')
for symbol in symbol_list:
    file.write(symbol+'\n')
file.close()

In [20]:
# Write to tinydb

symbol_db = TinyDB('symbol_db.json', storage=CachingMiddleware(JSONStorage))

for index, row in tqdm(symbol_df.iterrows()):
    symbol_db.insert({'Id': index, 
                      'Symbol': row['Symbol'], 
                      'Name': row['Name'], 
                      'Currency': row['Currency'], 
                      'Exchange': row['Exchange'],
                      'Status': 'untouched'
                     })
symbol_db.close()